In [ ]:
import numpy as np
np.random.seed(seed=484)
import warnings
warnings.filterwarnings('ignore')

# Midterm Exam

Exam is open book, open note, and open Google. You are not allowed outside
help from another person, however. All work, including coding, must be yours alone. Remember to turn in both the written portion and this coding portion. The coding portion can be turned in by submitting a shared link to your Colab notebook. To complete this coding portion, make sure to save a copy of this notebook in your own Google drive, supply the python code in the empty cells below, and execute the notebook. To get full credit, the completed notebook should be able to run top to bottom, producing the results asked for in the prompts below.

This portion of the exam will take you through the steps of the supervised machine learning process.

## 1. Figure out your question

The question you want to answer is: How does childbearing impact labor market outcomes for women? We can use machine learning to help answer this question by building a model that predicts how many children a woman gives birth to on the basis of her characteristics.

## 2. Obtain a labeled dataset

Import the python library that is good for manipulating datasets:

In [ ]:
import pandas as pd


Accompanying the exam materials are a spreadsheet of female survey respondents, 'femalelaborsupply.csv' and a text file, 'femalelaborsupplydefs.txt' that explains each variable in the spreadsheet. Read in the data in the spreadsheet 'femalelaborsupply.csv', print out the first few rows of data with the variable names, and print out the number of observations and variables in the dataset:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Econ\ 484

df = pd.read_csv('femalelaborsupply.csv')
print(df.head())
print("Shape (by observations and variables respectively): {}".format(str(df.shape)))

Mounted at /content/drive
/content/drive/MyDrive/Econ 484
   asex  aage  aqtrbrth  ageqk  ...   nonmomil  qobm  const  msample
0     0     0         0     36  ...  10.422200     2      1        1
1     0     0         0     23  ...   8.122422     3      1        0
2     0     0         0     44  ...  10.103316     4      1        1
3     0     0         0     24  ...  10.330663     3      1        1
4     0     0         0     28  ...  10.702120     1      1        1

[5 rows x 49 columns]
Shape (by observations and variables respectively): (394840, 49)


Define a label (outcome) vector, $y_1$, to be how many children the woman has, another outcome vector, $y_2$ to be an indicator for having three or more children, and define a feature (regressor) matrix, $X$, to contain the mother's age, marital status, race, ethnicity, and education:

In [ ]:
df = df.astype({"kidcount": int})

y1 = df.loc[:,'kidcount']

y2 = np.where(df['kidcount']>= 3, True, False)

#df.y2.head()

X = df.loc[:,['agem', 'marital', 'blackm', 'hispm', 'othracem', 'educm']]
#print(y1)
#print(y2)
#print(X)

"Pre-process" your features, $X$, by standardizing them to have zero mean and unit variance. Hint: you may import a useful package to do this.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
x_scaled = scaler.transform(X)
x_scaled.shape
#print(x_scaled)

(394840, 6)

## 3. Divide into training and set sets

Import the python library that is good for randomly splitting datasets into training and test sets:

In [ ]:
from sklearn.model_selection import train_test_split

Now make a training and test feature matrix and a training and test label vectors:

In [ ]:
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(x_scaled, y1, y2, random_state=42)
n_test=len(y2_test)
print(n_test)
print(len(y2_train)) 


98710
296130


## 4. Pick an appropriate method

Choose a method appropriate for classification and import its library:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

## 5 and 6. Choose regularization parameters via cross-validation on the training set and fit model on the whole training set using the cross-validated parameters

The outcome you should use in this part is $y_2$, the indicator for having at least three kids

Search over a grid of values of the regularization parameters for the parameters that perform the best on the left-out folds:

In [ ]:
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y2_train)

DecisionTreeClassifier(random_state=0)

## 7. Evaluate model by applying it to test set

Compute and print out the "score" of the model applied to the test set:

In [ ]:
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y2_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y2_test)))
tree1 = DecisionTreeClassifier(max_depth=5, min_samples_leaf = 18,random_state=0)
tree1.fit(X_train, y2_train)
# A version that performs better than the default:
print("Accuracy on training set: {:.3f}".format(tree1.score(X_train, y2_train)))
print("Accuracy on test set: {:.3f}".format(tree1.score(X_test, y2_test)))

Accuracy on training set: 0.637
Accuracy on test set: 0.629
Accuracy on training set: 0.631
Accuracy on test set: 0.630


## 8. Repeat 4-7 for $y_1$
using a method appropriate for regression-style prediction to predict number of children, not the probability of having at least three children

Import the method's library, do cross validation to find tuning parameters, fit the model on the training data using the cross-validated tuning parameters, and compute (and report) the model's score on the test set:

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_num = DecisionTreeRegressor(random_state=0)
tree_num.fit(X_train, y1_train)

print("Accuracy on training set: {:.3f}".format(tree_num.score(X_train, y1_train)))
print("Accuracy on test set: {:.3f}".format(tree_num.score(X_test, y1_test)))

Accuracy on training set: 0.112
Accuracy on test set: 0.077


## 9. Apply the prediction  models to new observations for which we have no labels

The spreadsheet 'newfemales.csv' contains information on two new females, with identical characteristics, except one is a high school graduate, and the other has a bachelor's degree.

Read in the new observations' information and apply the models to predict the probability of each applicant having at least three kids, and the predicted number of kids each applicant will have, and print out the predictions. Hint: don't forget to apply the same pre-processing steps to the new observations as you did to your training and test observations. This means standardizing the new observations using the means and variances of your labeled dataset, not the means and variances of these two new observations.

In [ ]:
new_df = pd.read_csv('newfemales.csv')
print(new_df.head())
print("Shape (by observations and variables respectively): {}".format(str(new_df.shape)))
list(new_df.columns)


   agem  marital  yobm  blackm  hispm  ...  agefstm  agefstd  qobm  const  msample
0    27        0    52       0      0  ...       18       26     2      1        1
1    27        0    52       0      0  ...       18       26     2      1        1

[2 rows x 21 columns]
Shape (by observations and variables respectively): (2, 21)


['agem',
 'marital',
 'yobm',
 'blackm',
 'hispm',
 'othracem',
 'blackd',
 'hispd',
 'othraced',
 'educm',
 'hsgrad',
 'hsormore',
 'moreths',
 'agem1',
 'aged1',
 'ageqm',
 'agefstm',
 'agefstd',
 'qobm',
 'const',
 'msample']

In [ ]:
new_X = new_df.loc[:,['agem', 'marital', 'blackm', 'hispm', 'othracem', 'educm']]
print(new_X)

   agem  marital  blackm  hispm  othracem  educm
0    27        0       0      0         0     12
1    27        0       0      0         0     16


In [ ]:
new_x_scaled = scaler.transform(new_X)
new_x_scaled.shape
print(new_x_scaled)

[[-0.89033895 -0.39320135 -0.3674086  -0.175984   -0.17184926 -0.05236481]
 [-0.89033895 -0.39320135 -0.3674086  -0.175984   -0.17184926  1.61389005]]


In [ ]:
#print("Accuracy on test set: {:.3f}".format(tree.score(new_x_scaled, y2_test)))
probability = tree.predict_proba(new_x_scaled)
probability = probability[:,1]

print("Probability that the HS Grad has at least 3 children: {:.5f}".format(probability[0]))
print("Probability that the applicant with the Bachelor's Degree has at least 3 children: {:.5f}".format(probability[1]))


Probability that the HS Grad has at least 3 children: 0.31596
Probability that the applicant with the Bachelor's Degree has at least 3 children: 0.26351


In [ ]:
numchildren = tree_num.predict(new_x_scaled)
numchildren
print("Predicted number of children for the applicant that is a HS Grad: {:.5f}".format(numchildren[0]))
print("Predicted number of children for the applicant that has a Bachelor's Degree: {:.5f}".format(numchildren[1]))


Predicted number of children for the applicant that is a HS Grad: 2.36876
Predicted number of children for the applicant that has a Bachelor's Degree: 2.28829


In [ ]:
# As we can see, the predictions line up - the applicant that is a HS Grad is predicted to be slightly more likely to have at least 3 children, 
# and has a slightly higher prediction for number of kids - relative to the similar applicant with a Bachelor's degree.